<a href="https://colab.research.google.com/github/Joyuri1022/Cloud_Network_Group2/blob/main/DATS6450_Final_Project_Modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
from typing import Dict, Any, List, Tuple
from huggingface_hub import login
import numpy as np
import torch
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import userdata

from tqdm import tqdm
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    AutoModelForCausalLM,
    AutoTokenizer,
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USE_COSINE = True
DEFAULT_THRESHOLD = 0.8

OUTPUT_DIR = '/content/drive/My Drive/Colab Notebooks/DATS6450_Network'
os.makedirs(OUTPUT_DIR, exist_ok=True)

CORPUS: List[str] = [
    "The cat sat on the mat.",
    "Climate change is a global challenge that requires urgent action.",
    "Quantum computing may revolutionize cryptography and optimization.",
    "The universe is vast and full of mysteries we have yet to uncover.",
    "Artificial intelligence can help us analyze large datasets efficiently.",
    "DNA encodes the genetic information necessary for life.",
    "The stock market fluctuates based on investor expectations and news.",
    "The brain is composed of billions of neurons forming complex networks.",
    "Mathematics provides a language to describe patterns in nature.",
    "Music can evoke powerful emotions and memories in people.",
    "For centuries, people believed that the Earth was flat, "
    "but modern science has shown with overwhelming evidence that our planet "
    "is an oblate spheroid, orbiting the Sun together with other planets."
]


MODEL_SPECS = [
    {
        "id": "gpt2",
        "hf_name": "gpt2",
        "model_class": GPT2LMHeadModel,
        "tokenizer_class": GPT2Tokenizer,
        "max_length": 40,
        "trust_remote_code": False,
    },
    {
        "id": "qwen1_5",
        "hf_name": "Qwen/Qwen1.5-0.5B",
        "model_class": AutoModelForCausalLM,
        "tokenizer_class": AutoTokenizer,
        "max_length": 40,
        "trust_remote_code": True,
    },
    {
        "id": "qwen2_5",
        "hf_name": "Qwen/Qwen2.5-0.5B",
        "model_class": AutoModelForCausalLM,
        "tokenizer_class": AutoTokenizer,
        "max_length": 40,
        "trust_remote_code": True,
    },
    {
        "id": "qwen2_5_instruct",
        "hf_name": "Qwen/Qwen2.5-0.5B-Instruct",
        "model_class": AutoModelForCausalLM,
        "tokenizer_class": AutoTokenizer,
        "max_length": 40,
        "trust_remote_code": True,
    },
    {
        "id": "qwen3",
        "hf_name": "Qwen/Qwen3-0.6B-base",
        "model_class": AutoModelForCausalLM,
        "tokenizer_class": AutoTokenizer,
        "max_length": 40,
        "trust_remote_code": True,
    },
]



In [7]:
# Load models and extract per layer tokens and compute cosine similarity
def load_model_and_tokenizer_from_spec(
    spec: Dict[str, Any],
    device: str = DEVICE,
):
    print(f"Loading model: {spec['hf_name']} on {device}...")
    tokenizer = spec["tokenizer_class"].from_pretrained(
        spec["hf_name"],
        trust_remote_code=spec.get("trust_remote_code", False),
    )
    model = spec["model_class"].from_pretrained(
        spec["hf_name"],
        output_hidden_states=True,
        trust_remote_code=spec.get("trust_remote_code", False),
    ).to(device)
    model.eval()

    return model, tokenizer

def extract_hidden_states(
    model,
    tokenizer,
    text: str,
    max_length: int = 40,
    device: str = DEVICE,
) -> Tuple[List[np.ndarray], List[str]]:
    """
    Run the model on a single text and return:
    - hidden_arrays: list of [seq_len, hidden_dim] per layer
    - tokens: list of token strings
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    hidden_states = outputs.hidden_states

    hidden_arrays = [h[0].cpu().numpy() for h in hidden_states]  # [seq_len, dim]
    token_ids = inputs["input_ids"][0]
    tokens = tokenizer.convert_ids_to_tokens(token_ids)

    return hidden_arrays, tokens


def compute_similarity_matrix(
    hidden_state: np.ndarray,
    use_cosine: bool = True,
) -> np.ndarray:
    """
    hidden_state: [seq_len, hidden_dim]
    return: [seq_len, seq_len] similarity matrix
    """
    if use_cosine:
        norms = np.linalg.norm(hidden_state, axis=1, keepdims=True) + 1e-9
        hs_norm = hidden_state / norms
        sim = hs_norm @ hs_norm.T
    else:
        sim = hidden_state @ hidden_state.T
    return sim

In [9]:
# Save similarity matrices per model into a single .npz file
for spec in MODEL_SPECS:
    model_id = spec["id"]
    print(f"\n=== Running model: {model_id} ===")
    model, tokenizer = load_model_and_tokenizer_from_spec(spec)

    # For each text in CORPUS, we will store:
    # - sims_per_text: list of [num_layers, seq_len, seq_len] arrays (one per text)
    # - tokens_per_text: list of token lists (one per text)
    sims_per_text: List[np.ndarray] = []
    tokens_per_text: List[List[str]] = []

    for text in CORPUS:
        hidden_states, tokens = extract_hidden_states(
            model,
            tokenizer,
            text,
            max_length=spec["max_length"],
            device=DEVICE,
        )

        # Compute similarity matrix for each layer
        sims_layers = []
        for layer in range(len(hidden_states)):
            sim_layer = compute_similarity_matrix(
                hidden_states[layer],
                use_cosine=USE_COSINE,
            )
            sims_layers.append(sim_layer)

        # Stack layers into [num_layers, seq_len, seq_len]
        sims_layers = np.stack(sims_layers, axis=0)
        sims_per_text.append(sims_layers)
        tokens_per_text.append(tokens)

        print(
            f"  text length={len(tokens):2d}, sims shape={sims_layers.shape}"
        )

    # Convert lists to explicit object arrays to avoid broadcasting issues
    sims_array = np.empty(len(sims_per_text), dtype=object)
    tokens_array = np.empty(len(tokens_per_text), dtype=object)
    for i in range(len(sims_per_text)):
        sims_array[i] = sims_per_text[i]       # [num_layers, seq_len_i, seq_len_i]
        tokens_array[i] = tokens_per_text[i]   # list of tokens for that text

    # Save everything for this model into a single .npz file
    save_path = os.path.join(OUTPUT_DIR, f"sims_{model_id}.npz")
    np.savez(
        save_path,
        sims=sims_array,                       # one entry per text
        tokens=tokens_array,                   # tokens per text
        corpus=np.array(CORPUS, dtype=object),
        model_id=np.array(model_id),
    )
    print(f"Saved similarity matrices for {model_id} to {save_path}")

    # Optional: free GPU memory in Colab
    del model, tokenizer
    if DEVICE == "cuda":
        torch.cuda.empty_cache()



=== Running model: gpt2 ===
Loading model: gpt2 on cpu...
  text length= 7, sims shape=(13, 7, 7)
  text length=11, sims shape=(13, 11, 11)
  text length=10, sims shape=(13, 10, 10)
  text length=14, sims shape=(13, 14, 14)
  text length=11, sims shape=(13, 11, 11)
  text length=10, sims shape=(13, 10, 10)
  text length=12, sims shape=(13, 12, 12)
  text length=12, sims shape=(13, 12, 12)
  text length=12, sims shape=(13, 12, 12)
  text length=10, sims shape=(13, 10, 10)
  text length=38, sims shape=(13, 38, 38)
Saved similarity matrices for gpt2 to /content/drive/My Drive/Colab Notebooks/DATS6450_Network/sims_gpt2.npz

=== Running model: qwen1_5 ===
Loading model: Qwen/Qwen1.5-0.5B on cpu...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

  text length= 7, sims shape=(29, 7, 7)
  text length=11, sims shape=(29, 11, 11)
  text length=10, sims shape=(29, 10, 10)
  text length=14, sims shape=(29, 14, 14)
  text length=11, sims shape=(29, 11, 11)
  text length=10, sims shape=(29, 10, 10)
  text length=12, sims shape=(29, 12, 12)
  text length=12, sims shape=(29, 12, 12)
  text length=11, sims shape=(29, 11, 11)
  text length=10, sims shape=(29, 10, 10)
  text length=39, sims shape=(29, 39, 39)
Saved similarity matrices for qwen3 to /content/drive/My Drive/Colab Notebooks/DATS6450_Network/sims_qwen3.npz


# Experiment 1
## Corpus Average Percolation (Using same threshold across models)

In [13]:
def build_connectivity_graph(
    similarity_matrix: np.ndarray,
    threshold: float,
    tokens: List[str],
) -> nx.Graph:
    """
    Build an undirected graph where nodes are tokens,
    and edges connect pairs with similarity > threshold.
    """
    n = len(tokens)
    G = nx.Graph()
    for i in range(n):
        G.add_node(i, token=tokens[i])

    for i in range(n):
        for j in range(i + 1, n):
            sim_ij = similarity_matrix[i, j]
            if sim_ij > threshold:
                G.add_edge(i, j, weight=float(sim_ij))

    return G


def analyze_graph_structure(G: nx.Graph) -> Dict[str, Any]:
    """
    Return percolation-style observables:
    - component_sizes
    - largest_component_size
    - phi: largest_component_size / num_nodes
    - num_clusters
    - chi: mean cluster size excluding GCC
    """
    num_nodes = G.number_of_nodes()
    if num_nodes == 0:
        return {
            "component_sizes": [],
            "largest_component_size": 0,
            "phi": 0.0,
            "num_clusters": 0,
            "chi": 0.0,
        }

    components = list(nx.connected_components(G))
    component_sizes = [len(c) for c in components]
    largest_component_size = max(component_sizes)

    phi = largest_component_size / num_nodes
    num_clusters = len(component_sizes)

    finite_sizes = [s for s in component_sizes if s < largest_component_size]
    if len(finite_sizes) > 0:
        chi = float(np.mean(finite_sizes))
    else:
        chi = 0.0

    return {
        "component_sizes": component_sizes,
        "largest_component_size": largest_component_size,
        "phi": float(phi),
        "num_clusters": num_clusters,
        "chi": chi,
    }


def visualize_network(
    G: nx.Graph,
    tokens: List[str],
    layer_num: int,
    title: str = "",
    ax=None,
    layout: str = "spring",
):
    """
    Draw a token similarity graph with connected components colored differently.
    """
    if ax is None:
        ax = plt.gca()

    if len(G) == 0:
        ax.set_axis_off()
        ax.set_title(f"Layer {layer_num}: Empty graph")
        return

    if layout == "spring":
        pos = nx.spring_layout(G, seed=42, k=0.5)
    elif layout == "kamada_kawai":
        pos = nx.kamada_kawai_layout(G)
    else:
        pos = nx.spring_layout(G, seed=42)

    components = list(nx.connected_components(G))
    component_sizes = [len(c) for c in components]
    largest_size = max(component_sizes)

    color_map = []
    for node in G.nodes():
        comp_idx = None
        for i, comp in enumerate(components):
            if node in comp:
                comp_idx = i
                break
        size = component_sizes[comp_idx]
        if size == largest_size and largest_size > 1:
            color_map.append("tab:red")
        else:
            color_map.append("tab:blue")

    nx.draw_networkx_nodes(G, pos, node_size=400, node_color=color_map, ax=ax)
    nx.draw_networkx_edges(G, pos, alpha=0.4, ax=ax)
    labels = {i: tokens[i] for i in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=8, ax=ax)

    stats = analyze_graph_structure(G)
    phi = stats["phi"]
    num_clusters = stats["num_clusters"]
    largest_component_size = stats["largest_component_size"]
    n_nodes = G.number_of_nodes()

    if not title:
        title = (
            f"Layer {layer_num} | GCC: {largest_component_size}/{n_nodes} "
            f"(phi={phi:.2f}), clusters={num_clusters}"
        )
    ax.set_title(title, fontsize=9)
    ax.set_axis_off()


In [12]:
import numpy as np

path = "/content/drive/My Drive/Colab Notebooks/DATS6450_Network/sims_gpt2.npz"
data = np.load(path, allow_pickle=True)

sims = data["sims"]        # array of length = number of texts
tokens = data["tokens"]    # array of token lists
corpus = data["corpus"]    # array of raw text strings
model_id = data["model_id"]  # string

text_idx = -1
sims_text = sims[text_idx]          # shape: (num_layers, seq_len, seq_len)
num_layers = sims_text.shape[0]

for layer_idx in range(num_layers):
    sim_layer = sims_text[layer_idx]   # (seq_len, seq_len)
    print(f"Layer {layer_idx}, sim shape {sim_layer}")

Layer 0, sim shape [[1.0000001  0.45204747 0.43217477 ... 0.33540887 0.2894737  0.3023032 ]
 [0.45204747 1.0000001  0.6586286  ... 0.36430156 0.4012145  0.33827516]
 [0.43217477 0.6586286  1.0000002  ... 0.43304598 0.28474697 0.5998092 ]
 ...
 [0.33540887 0.36430156 0.43304598 ... 0.9999999  0.635992   0.7331997 ]
 [0.2894737  0.4012145  0.28474697 ... 0.635992   1.         0.5826245 ]
 [0.3023032  0.33827516 0.5998092  ... 0.7331997  0.5826245  0.99999976]]
Layer 1, sim shape [[1.0000005  0.23767589 0.2825894  ... 0.14616401 0.14376299 0.25608456]
 [0.23767589 0.99999976 0.77100736 ... 0.56321865 0.6260421  0.6341916 ]
 [0.2825894  0.77100736 0.9999999  ... 0.6747431  0.582166   0.8482634 ]
 ...
 [0.14616401 0.56321865 0.6747431  ... 1.0000004  0.64717925 0.7440355 ]
 [0.14376299 0.6260421  0.582166   ... 0.64717925 0.9999998  0.6437883 ]
 [0.25608456 0.6341916  0.8482634  ... 0.7440355  0.6437883  1.0000004 ]]
Layer 2, sim shape [[0.9999993  0.08259699 0.08913913 ... 0.0608815  0.066

In [14]:
import os
import numpy as np
from typing import Any, Dict, List, Tuple


def load_similarity_cache(
    model_id: str,
    npz_dir: str,
) -> Tuple[List[str], List[List[str]], List[np.ndarray]]:
    """
    Load cached similarity data for a given model.

    Returns:
      - corpus: list of raw text strings
      - tokens_list: list of token lists, one per text
      - sims_list: list of numpy arrays, one per text
        each sims_list[i] has shape [num_layers, seq_len_i, seq_len_i]
    """
    path = os.path.join(npz_dir, f"sims_{model_id}.npz")
    data = np.load(path, allow_pickle=True)

    corpus_arr = data["corpus"]   # object array
    tokens_arr = data["tokens"]   # object array
    sims_arr = data["sims"]       # object array of 3D arrays

    corpus: List[str] = [str(t) for t in corpus_arr]
    tokens_list: List[List[str]] = [list(tok_seq) for tok_seq in tokens_arr]
    sims_list: List[np.ndarray] = [np.array(s) for s in sims_arr]

    print(f"Loaded cache for model={model_id}")
    print(f"Number of texts: {len(corpus)}")
    print(f"Layers in first text: {sims_list[0].shape[0]}")

    return corpus, tokens_list, sims_list

def compute_percolation_stats_from_sims(
    tokens_list: List[List[str]],
    sims_list: List[np.ndarray],
    threshold: float,
) -> Tuple[List[Dict[str, Any]], Dict[int, Dict[str, float]]]:
    """
    Given cached sims and tokens, compute per-layer percolation stats.

    Returns:
      - all_results: list over texts, each:
            {
              "text": <optional, you can attach later>,
              "tokens": tokens_for_text,
              "layers": [
                  { "graph": G, "stats": stats_dict }, ...
              ]
            }
      - aggregate_stats: dict[layer] -> {
              "mean_phi", "std_phi",
              "mean_num_clusters", "std_num_clusters",
              "mean_chi", "std_chi"
        }
    """
    num_texts = len(sims_list)
    assert num_texts == len(tokens_list), "tokens_list and sims_list length mismatch"

    num_layers = sims_list[0].shape[0]
    print(f"Computing percolation stats for {num_texts} texts, {num_layers} layers")

    all_results: List[Dict[str, Any]] = []

    # - Per text, per layer
    for text_idx in range(num_texts):
        sims_text = sims_list[text_idx]          # [num_layers, seq_len, seq_len]
        tokens = tokens_list[text_idx]
        seq_results = {
            "tokens": tokens,
            "layers": [],
        }

        for layer in range(num_layers):
            sim_mat = sims_text[layer]           # [seq_len, seq_len]
            G = build_connectivity_graph(sim_mat, threshold, tokens)
            stats = analyze_graph_structure(G)

            seq_results["layers"].append(
                {
                    "graph": G,
                    "stats": stats,
                }
            )

        all_results.append(seq_results)

    # - Aggregate across texts, per layer
    aggregate_stats: Dict[int, Dict[str, float]] = {}

    for layer in range(num_layers):
        phis = [res["layers"][layer]["stats"]["phi"] for res in all_results]
        num_clusters = [res["layers"][layer]["stats"]["num_clusters"] for res in all_results]
        chis = [res["layers"][layer]["stats"]["chi"] for res in all_results]

        aggregate_stats[layer] = {
            "mean_phi": float(np.mean(phis)),
            "std_phi": float(np.std(phis)),
            "mean_num_clusters": float(np.mean(num_clusters)),
            "std_num_clusters": float(np.std(num_clusters)),
            "mean_chi": float(np.mean(chis)),
            "std_chi": float(np.std(chis)),
        }

    return all_results, aggregate_stats


In [24]:
import numpy as np
from typing import Dict, Optional

def find_critical_layer(
    aggregate_stats: Dict[int, Dict[str, float]],
    phi_key: str = "mean_phi",
    phi_threshold: float = 0.5,
    z_cut: float = 1.0,
) -> Optional[int]:
    """
    Critical layer: layer l where
      - jump dphi(l) is "large": dphi(l) >= mean(dphi) + z_cut * std(dphi)
      - phi(l) >= phi_threshold
    Among all such layers, return the one with the largest dphi.
    """
    if not aggregate_stats:
        return None

    layers = sorted(aggregate_stats)
    phi = np.array([aggregate_stats[l][phi_key] for l in layers], float)

    if len(phi) < 2:
        return layers[0] if phi[0] >= phi_threshold else None

    # jumps from previous layer
    dphi = np.diff(phi)
    if np.all(dphi <= 0):
        return None

    # "large" jump threshold
    mean_dphi, std_dphi = dphi.mean(), dphi.std()

    # fallback when jumps are almost all the same
    if std_dphi < 1e-8:
        j = int(np.argmax(dphi))   # index in dphi
        upper_idx = j + 1          # index in phi and layers
        if dphi[j] > 0 and phi[upper_idx] >= phi_threshold:
            return layers[upper_idx]
        return None

    jump_min = mean_dphi + z_cut * std_dphi

    # candidate layers = upper endpoint of jump
    candidates = [
        i + 1 for i, dj in enumerate(dphi)
        if dj >= jump_min and phi[i + 1] >= phi_threshold
    ]
    if not candidates:
        return None

    # choose candidate with largest jump
    best = max(candidates, key=lambda idx: dphi[idx - 1])
    return layers[best]


In [25]:
model_id_list = ["gpt2", "qwen1_5", "qwen2_5", "qwen2_5_instruct", "qwen3"]


threshold_list = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]

for threshold in threshold_list:
    for model_id in model_id_list:

        print(f"\n=== MODEL: {model_id}, THRESHOLD: {threshold} ===")

        # 1. Load cached sims
        corpus, tokens_list, sims_list = load_similarity_cache(
            model_id=model_id,
            npz_dir=OUTPUT_DIR,
        )

        # 2. Compute percolation stats
        all_results, aggregate_stats = compute_percolation_stats_from_sims(
            tokens_list=tokens_list,
            sims_list=sims_list,
            threshold=threshold,
        )

        critical_layer = find_critical_layer(
            aggregate_stats,
            phi_key="mean_phi",
            phi_threshold=0.7,  # "GCC emergence" threshold
            z_cut=1.5,          # 1 std above mean jump; you can try 0.5 or 1.5 to see robustness
        )
        print(f"Critical Layer (model={model_id}, threshold={threshold}): {critical_layer}")

        # 3. Print summary table
        num_layers = len(aggregate_stats)

        print("\n" + "=" * 70)
        print(f"SUMMARY STATISTICS ACROSS ALL SEQUENCES - {model_id}, threshold={threshold}")
        print("=" * 70)
        print(f"{'Layer':<8} {'Phi (GCC)':<16} {'#Clusters':<15} {'Chi (Mean Size)':<15}")
        print("-" * 70)

        for layer in range(num_layers):
            phi = aggregate_stats[layer]["mean_phi"]
            phi_std = aggregate_stats[layer]["std_phi"]
            nc = aggregate_stats[layer]["mean_num_clusters"]
            nc_std = aggregate_stats[layer]["std_num_clusters"]
            chi = aggregate_stats[layer]["mean_chi"]
            chi_std = aggregate_stats[layer]["std_chi"]

            print(
                f"{layer:<8} {phi:.3f}±{phi_std:.3f}   "
                f"{nc:.1f}±{nc_std:.1f}         "
                f"{chi:.2f}±{chi_std:.2f}"
            )

        print("=" * 70)


=== MODEL: gpt2, THRESHOLD: 0.5 ===
Loaded cache for model=gpt2
Number of texts: 11
Layers in first text: 13
Computing percolation stats for 11 texts, 13 layers
Critical Layer (model=gpt2, threshold=0.5): None

SUMMARY STATISTICS ACROSS ALL SEQUENCES - gpt2, threshold=0.5
Layer    Phi (GCC)        #Clusters       Chi (Mean Size)
----------------------------------------------------------------------
0        0.912±0.026   2.0±0.0         1.00±0.00
1        0.912±0.026   2.0±0.0         1.00±0.00
2        0.912±0.026   2.0±0.0         1.00±0.00
3        0.929±0.046   1.7±0.4         0.73±0.45
4        0.946±0.052   1.5±0.5         0.55±0.50
5        0.973±0.039   1.4±0.5         0.36±0.48
6        0.998±0.008   1.1±0.3         0.09±0.29
7        0.998±0.008   1.1±0.3         0.09±0.29
8        0.998±0.008   1.1±0.3         0.09±0.29
9        0.998±0.008   1.1±0.3         0.09±0.29
10       0.998±0.008   1.1±0.3         0.09±0.29
11       0.974±0.043   1.3±0.4         0.27±0.45
12       